# CartoCamp: Data Science + Maps workshop

### Jan 27, 2017

* Andy Eschbacher, map scientist, CARTO, [@MrEPhysics](https://twitter.com/MrEPhysics), andy@carto.com
* Danny Sheehan, solutions engineer, CARTO, [@nygeog](https://twitter.com/nygeog), danny@carto.com

Event page: https://www.meetup.com/CartoCamp/events/236668763/

## Why we're having this workshop

We've been frustrated by the pain points in working including maps in the data science workflow, especially interactive maps. While we don't have a full solution, we're going to show you some of the tricks and tips we've picked up along the way. We also hope that you can share what you've learned here and open a pull request against this repo to include things that we left out!

**Our goal** today is to **analyze spatial data** and **visualize it (i.e., create maps)** in a standard data science workflow and toolkit (Jupyter notebook + pandas, scikit-learn, requests, NumPy, ...). We'll be using CARTO for some of the spatial processing and map generation.



In [1]:
import IPython
iframe = '<iframe width="100%" height="520" frameborder="0" src="https://team.carto.com/u/eschbacher/builder/96da150a-b8c9-11e6-8a45-0ecd1babdde5/embed" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>'
IPython.display.HTML(iframe)

## Getting setup

* Development environment: check out the [README](https://github.com/CartoCamp/workshops/tree/master/2017-01-27-data-sci-maps/README.md) to get up and going!
* Get a CARTO account

## Grabbing the taxi data

**BEWARE** there are around 11 million entries in this dataset (which is around 1 GB), so make sure to subsample the pieces you want. Using WHERE filters or LIMITs in SQL are the best way to get subsamples.

This dataset is stored in my CARTO account, and we'll access it via CARTO's [SQL API](https://carto.com/docs/carto-engine/sql-api).

In [2]:
import pandas as pd
import numpy as np

NUMROWS = 50000
DAY = '2015-08-22'
cred = json.load(open('credentials.json')) # modify credentials.json.sample

username = cred['username']
api_key  = cred['api_key']

sql_api_template = 'https://{username}.carto.com/api/v2/sql?q={query}&format=csv'

def uri_query(query):
    """
        Prepare SQL queries for API usage
    """
    return query.strip().replace('\n', ' ').replace(' ', '%20')

### Understanding our data

In [3]:
query = '''
SELECT * FROM taxi_aug15_sept15
LIMIT 10
'''

sql_api_request = sql_api_template.format(query=uri_query(query),
                                          username=username)
df = pd.read_csv(sql_api_request).set_index('cartodb_id')

df.head()

,the_geom,the_geom_webmercator,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,ratecodeid,...,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
cartodb_id,,,,,,,,,,,,,,,,,,,,,
57444070,NaN,NaN,1,2015-08-16 21:46:13+00,2015-08-16 21:49:45+00,1,0.80,-73.984764,40.732044,1,...,-73.996231,40.738121,1,4.5,0.5,0.5,1.0,0.0,0.3,6.8
57443485,NaN,NaN,1,2015-08-16 21:34:51+00,2015-08-16 21:37:31+00,1,0.40,-73.979912,40.759033,1,...,-73.977104,40.764370,2,4.0,0.5,0.5,0.0,0.0,0.3,5.3
57443486,NaN,NaN,1,2015-08-16 21:34:51+00,2015-08-16 21:40:40+00,1,1.00,-73.984917,40.760937,1,...,-73.977905,40.753777,2,6.0,0.5,0.5,0.0,0.0,0.3,7.3
57443487,NaN,NaN,1,2015-08-16 21:34:51+00,2015-08-16 21:42:02+00,1,0.80,-73.999802,40.728455,1,...,-73.994766,40.730343,2,6.5,0.5,0.5,0.0,0.0,0.3,7.8
57443488,NaN,NaN,2,2015-08-16 21:34:51+00,2015-08-16 22:01:42+00,1,5.86,-73.971428,40.744301,1,...,-73.964188,40.679691,2,22.0,0.5,0.5,0.0,0.0,0.3,23.3


In [23]:
df.columns

Index([u'the_geom', u'the_geom_webmercator', u'vendorid',
       u'tpep_pickup_datetime', u'tpep_dropoff_datetime', u'passenger_count',
       u'trip_distance', u'pickup_longitude', u'pickup_latitude',
       u'ratecodeid', u'store_and_fwd_flag', u'dropoff_longitude',
       u'dropoff_latitude', u'payment_type', u'fare_amount', u'extra',
       u'mta_tax', u'tip_amount', u'tolls_amount', u'improvement_surcharge',
       u'total_amount'],
      dtype='object')

### Let's get a summary of the data

In [28]:
query = '''
SELECT 
    min(tpep_pickup_datetime) As min_date,
    max(tpep_pickup_datetime) As max_date,
    count(*) As n_trips,
    sum(passenger_count) As n_total_passengers,
    max(passenger_count) As max_n_passengers,
    avg(trip_distance) As avg_trip_distance,
    max(fare_amount) As max_fare,
    avg(fare_amount) As avg_fare,
    min(fare_amount) As min_fare,
    max(tip_amount) As max_tip_amount,
    avg(tip_amount) As avg_tip_amount,
    min(tip_amount) As min_tip_amount,
    count(tolls_amount) FILTER (WHERE tolls_amount > 0) n_trips_with_tolls
FROM 
    taxi_aug15_sept15
'''

summary = pd.read_csv(sql_api_template.format(query=uri_query(query), 
                                              username=username))
summary.head()

,min_date,max_date,n_trips,n_total_passengers,max_n_passengers,avg_trip_distance,max_fare,avg_fare,min_fare,max_tip_amount,avg_tip_amount,min_tip_amount,n_trips_with_tolls
0,2015-08-15 04:00:00+00,2015-09-16 03:59:59+00,11471615,19392566,9,8.428135,124518.47,13.12243,-400,800,1.671428,-100,637424


In [29]:
query = '''
SELECT 
    pickup_longitude as long,
    pickup_latitude as lat,
    passenger_count,
    fare_amount,
    total_amount
FROM 
    taxi_aug15_sept15
WHERE
    fare_amount > 1000
'''

high_rollers = pd.read_csv(api_template.format(query=uri_query(query), username=username))
out_name = 'high_rollers'
high_rollers.head()

,long,lat,passenger_count,fare_amount,total_amount
0,-73.971123,40.763165,1,4000.85,4001.35
1,-73.870979,40.773621,3,107399.55,107445.09
2,-74.013664,40.706070,2,2759.07,2759.37
3,-74.014252,40.706596,1,2759.07,2759.37
4,-74.014206,40.706814,1,2759.07,2759.37


In [34]:
def send_to_carto(df, tablename, n_iters=10):
    import requests
    import time
    import json
    filename = '{}.csv'.format(tablename)
    df.to_csv('{}.csv'.format(filename))
    cred = json.load(open('credentials.json')) # modify credentials.json.sample

    username = cred['username']
    api_key  = cred['api_key']
    api_template = "https://{username}.carto.com/api/v1/imports/?api_key={api_key}"
    files = {'file': open(filename , 'rb')}
    resp = requests.post(api_template.format(username=username, 
                                             api_key=api_key),
                         files=files)
    queue_id = json.loads(resp.text)['item_queue_id']
    print("item_queue_id: {}".format(queue_id))

    last_status = None
    for i in range(n_iters):
        time.sleep(2)
        api_call_status = 'https://{username}.carto.com/api/v1/imports/{queue_id}?api_key={api_key}'
        status = requests.get(api_call_status.format(username=username,
                                                     api_key=api_key,
                                                     queue_id=queue_id))
        if json.loads(status.text)['state'] != last_status:
            print("{tablename}: {state}".format(tablename=tablename,
                                               state=json.loads(status.text)['state']))
        last_status = json.loads(status.text)['state']
        if last_status in ('complete', 'failure'):
            break

In [35]:
send_to_carto(high_rollers, 'high_rollers')

item_queue_id: ba3dbb70-ce25-44c1-a53a-36b6e727f938
high_rollers: importing
high_rollers: complete


In [4]:
def get_carto_map(tablename, colname):
    import requests
    import json
    cred = json.load(open('credentials.json')) # modify credentials.json.sample

    username = cred['username']
    api_key  = cred['api_key']
    maps_api_call_template = "https://{username}.carto.com/api/v1/map/named?api_key={api_key}"
    headers = {'Content-Type': 'application/json'}
    mapconfig = '''
{
  "version": "0.0.1",
  "name": "andy_cartocamp_test2",
  "auth": {
      "method": "open"
  },
  "layergroup": {
    "layers": [
      {
        "type": "mapnik",
        "options": {
          "cartocss_version": "2.1.1",
          "cartocss": "#layer { marker-fill: #FFF; }",
          "sql": "select * from %(tablename)s"
        }
      }
    ]
  },
  "view": {
    "zoom": 12,
    "center": {
      "lng": -74.0059,
      "lat": 40.7127
    }
  }
}
    ''' % {'tablename': tablename}
    print(type(json.loads(mapconfig)))
    payload = {'data': json.dumps(json.loads(mapconfig))}
    print(json.loads(mapconfig))
    print (maps_api_call_template.format(username=username, api_key=api_key))
    resp = requests.post(maps_api_call_template.format(username=username,
                                                       api_key=api_key),
                         headers=headers,
                         data=json.dumps(payload))
    print(resp.text)

In [5]:
get_carto_map('high_rollers', 'passenger_count')

<type 'dict'>
{u'version': u'0.0.1', u'view': {u'zoom': 12, u'center': {u'lat': 40.7127, u'lng': -74.0059}}, u'name': u'andy_cartocamp_test2', u'auth': {u'method': u'open'}, u'layergroup': {u'layers': [{u'type': u'mapnik', u'options': {u'cartocss': u'#layer { marker-fill: #FFF; }', u'sql': u'select * from high_rollers', u'cartocss_version': u'2.1.1'}}]}}
https://eschbacher.carto.com/api/v1/map/named?api_key=05a458d3a45d1237699a4ee05297bb92accce3f4
{"errors":["Unsupported template version undefined"],"errors_with_context":[{"type":"unknown","message":"Unsupported template version undefined"}]}
